# Projective Transformations

**Goal**: Given four markers  in view, find out where within this area the DIPPID device (Smartphone or Webcam) is pointing at.

**Reminder**: Projective Transforms do not preserve parallelism, length and angle (contrary to affine transformations), yet preserve collinearity and incidence 

**See**: Homography, Vector Spaces

In [ ]:
%pylab inline
%config InlineBackend.figure_format='svg'

In [ ]:
CAM_WIDTH, CAM_HEIGHT = 1024, 768 # the dimensions 

In [ ]:
# Four markers indicating the corners of the display with which we interact

# needs some more pre-processing in practical usage
# sorting of points required 
# define which one is the top left corner, bottom left corner, ...

A = 450, 690
B = 500, 300
C = 950, 300
D = 900, 700

scoords = [A, B, C, D]
scoords

In [ ]:
xlim(0, CAM_WIDTH)
ylim(0, CAM_HEIGHT)
scatter(*zip(*scoords)); # repack points[] to axes[] for plotting
scatter([CAM_WIDTH / 2],[CAM_HEIGHT / 2],c='r',marker='+'); # center of the Wiimote sensor i.e. the position we point to.

Our Goal: find out where our Wiimote is pointing at:
based on:
http://math.stackexchange.com/questions/296794/finding-the-transform-matrix-from-4-projected-points-with-javascript

**Step 1:** Starting with the 4 positions in the source image, named $A, B, C, D$, you solve the following system of linear equations:

$$\begin{pmatrix}
A_x & B_x & C_x \\
A_y & B_y & C_y \\
1 & 1 & 1
\end{pmatrix}\cdot
\begin{pmatrix}\lambda\\\mu\\\tau\end{pmatrix}=
\begin{pmatrix}D_x\\D_y\\1\end{pmatrix}$$

These are called homogenous coordinates. We fixed the basis of our vector space by using A, B and C. This basis allows us to represent any point within this vector space, in this case D. By solving this system of linear equations, we retrieve the values for lambda, my and tau. These are the factors we need to transform from the identity matrix to the source matrix.

In [ ]:
# Step 1
source_points_123 = matrix([[A[0], B[0], C[0]], 
                            [A[1], B[1], C[1]], 
                            [  1 ,   1 ,   1 ]])

source_point_4 = [[D[0]],
                  [D[1]],
                  [ 1 ]]

# solve the system of linear equations
scale_to_source = solve(source_points_123, source_point_4)

In [ ]:
scale_to_source

In [ ]:
l, m, t = [float(x) for x in scale_to_source]
l, m, t

**Step 2:** Scale the columns by the coefficients you just computed:

$$A=\left(\begin{array}{lll}
\lambda\cdot x_1 & \mu\cdot x_2 & \tau\cdot x_3 \\
\lambda\cdot y_1 & \mu\cdot y_2 & \tau\cdot y_3 \\
\lambda & \mu & \tau
\end{array}\right)$$

Now we computed the matrix, which transforms a Point P from the identity matrix to the given projection matrix.

In [ ]:
# Step 2
identity_to_source = matrix([[l * A[0], m * B[0], t * C[0]], 
                             [l * A[1], m * B[1], t* C[1]], 
                             [     l ,      m ,    t ]])

In [ ]:
identity_to_source

**Step 3:** Repeat steps 1 and 2 for the corresponding positions in the destination image (our unprojected, undistorted image), in order to obtain a second matrix called $B$.

This is a map from basis vectors to destination positions.

In [ ]:
# Step 3
DESTINATION_SCREEN_WIDTH = 1280
DESTINATION_SCREEN_HEIGHT = 720

A2 = 0, DESTINATION_SCREEN_HEIGHT
B2 = 0, 0
C2 = DESTINATION_SCREEN_WIDTH, 0
D2 = DESTINATION_SCREEN_WIDTH, DESTINATION_SCREEN_HEIGHT

dcoords = [A2, B2, C2, D2]

dest_points_123 = matrix([[A2[0], B2[0], C2[0]], 
                          [A2[1], B2[1], C2[1]], 
                          [  1 ,   1 ,   1 ]])
            
dest_point_4 = matrix([[D2[0]],
                       [D2[1]],
                       [ 1 ]])
            
scale_to_dest = solve(dest_points_123, dest_point_4)
l,m,t = [float(x) for x in scale_to_dest]

In [ ]:
identity_to_dest = matrix([[l * A2[0], m * B2[0], t * C2[0]], 
                           [l * A2[1], m * B2[1], t * C2[1]], 
                           [      l ,       m ,      t ]])

In [ ]:
identity_to_dest

**Step 4:** Invert $A$ to obtain $A^{-1}$.

By inverting the matrix A we transform from source to identity instead of identity to source. Therefore, it transforms a point P of the projection matrix to a point P' represented by the identity matrix.

In [ ]:
source_to_identity = inv(identity_to_source)

**Step 5:** Compute the combined matrix 

$C = B\cdot A^{-1}$.

* We multiply $B$ with $A^{-1}$. from the left, because matrix multiplications are not commutative. So if we want to combine these matrices and keep both of their transformation behaviours, we have to do it that way. 

* We basically push the desired point from the source matrix A to the unit matrix and then push it to the desired destination position according to the destination martix $B$

In [ ]:
source_to_dest = identity_to_dest @ source_to_identity

In [ ]:
source_to_dest

**Step 6:** To map a location $(x,y)$ from the source image to its corresponding location in the destination image, compute the product

$$\begin{pmatrix}x'\\y'\\z'\end{pmatrix} =
C\cdot\begin{pmatrix}x\\y\\1\end{pmatrix}$$

These are the homogenous coordinates of your transformed point.

In [ ]:
x,y,z = [float(w) for w in (source_to_dest @ matrix([[512],
                                                     [384],
                                                     [ 1 ]]))]

In [ ]:
x, y, z

**Step 7:** Compute the position in the destination image like this:

\begin{align*}
x'' &= \frac{x'}{z'} \\
y'' &= \frac{y'}{z'}
\end{align*}

This is called *dehomogenization* of the coordinate vector. In that way, we retrieve the actual image coordinates of our projected point in the desired destination image, so we can use them for e.g. pointing.

In [ ]:
# step 7: dehomogenization
x, y = x / z, y / z
x, y

In [ ]:
xlim(0, DESTINATION_SCREEN_WIDTH)
ylim(0, DESTINATION_SCREEN_HEIGHT)
scatter(*zip(*dcoords)) # repack points[] to axes[] for plotting
scatter([x],[y],c='r',marker='+') # center of the Wiimote sensor